In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import dateutil
import datetime
import re
import matplotlib.pyplot as plt
import math

import load_data

import pecos

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.palettes import Category10
from bokeh.layouts import gridplot
from bokeh.models import Legend
output_notebook()

# pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
#test

In [ ]:
raw_data = load_data.load_data(directory='./data_b/')

In [ ]:
raw_data.shape

In [ ]:
trans = load_data.trans_dict(raw_data)

In [ ]:
pm = pecos.monitoring.PerformanceMonitoring()
system_name = 'test'
pm.add_dataframe(raw_data, system_name)
pm.add_translation_dictionary(trans, system_name)

In [ ]:
grid = load_data.plot(pm)
show(grid)

In [ ]:
raw_data_sensors_filtered = load_data.apply_filter(pm, perc_diff=.05, skip_strs=['op_state', 'real_pwr-valuesError-inv-', 'wind'])

In [ ]:
raw_data_sensors_filtered.shape

In [ ]:
dtindex = pd.DatetimeIndex(freq='min', start=raw_data_sensors_filtered.index.tolist()[0], end=raw_data_sensors_filtered.index.tolist()[-1])

In [ ]:
filtered = raw_data_sensors_filtered.reindex(index=dtindex)

In [ ]:
trans2 = load_data.trans_dict(raw_data_sensors_filtered)

In [ ]:
pm2 = pecos.monitoring.PerformanceMonitoring()
pm2.add_dataframe(filtered, 'clean')
pm2.add_translation_dictionary(trans2, 'clean')

In [ ]:
grid2 = load_data.plot(pm2)
show(grid2)

In [ ]:
pm.check_timestamp(300)

In [ ]:
# for b
# pm.check_missing()
# pm.check_range([-5,1500], 'irr-ghi-pyran')
# pm.check_range([-5,1500], 'irr-poa-pyran')
# pm.check_range([-5,1500], 'irr-poa-ref_cell')
# pm.check_range([0,10], 'op_state-inv-')
# pm.check_range([-1,1], 'pf-inv')
# pm.check_range([-10,40], 'temp-amb-')
# pm.check_range([-10,127], 'temp-mod-')
# pm.check_range([0,18], 'wind--')

# for c
pm.check_missing()
pm.check_range([-5,1500], 'irr-ghi-')
pm.check_range([-5,1500], 'irr-poa-')
pm.check_range([-5,1500], 'irr-poa-ref_cell')
pm.check_range([-10,40], 'temp-amb-')
pm.check_range([0,18], 'wind--')

In [ ]:
check_range_keys = trans_keys.copy()

In [ ]:
check_range_keys

In [ ]:
check_range_keys.remove('real_pwr-valuesError-mtr-') 
check_range_keys.remove('real_pwr-valuesError-inv-')

In [ ]:
for key in check_range_keys:
    pm.check_increment([0.001, None], key)

In [ ]:
mask = pm.get_test_results_mask()
QCI = pecos.metrics.qci(mask)

In [ ]:
# Define output file names and directories
results_directory = 'Results'
if not os.path.exists(results_directory):
    os.makedirs(results_directory)
graphics_file_rootname = os.path.join(results_directory, 'test_results')
custom_graphics_file = os.path.abspath(os.path.join(results_directory, 'custom.png'))
metrics_file = os.path.join(results_directory, system_name + '_metrics.csv')
test_results_file = os.path.join(results_directory, system_name + '_test_results.csv')
report_file =  os.path.join(results_directory, system_name + '.html')

# Generate graphics
test_results_graphics = pecos.graphics.plot_test_results(graphics_file_rootname, pm)
fig = plt.figure(figsize = (7.0,3.5))

ax1 = fig.add_subplot(311)
pm.df[pm.trans['irr-poa-ref_cell']].plot()

plt.savefig(custom_graphics_file, format='png', dpi=500)

# Write metrics, test results, and report files
pecos.io.write_metrics(metrics_file, QCI)
pecos.io.write_test_results(test_results_file, pm.test_results)
pecos.io.write_monitoring_report(report_file, pm, test_results_graphics, [custom_graphics_file], QCI)